<a href="https://colab.research.google.com/github/jrsheffie/Text-Sentimization/blob/JS_POC/POC_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
#upload dataset
#!pip install -U datasets huggingface_hub fsspec

#!pip install datasets
#from datasets import load_dataset


#dataset = load_dataset("Exorde/exorde-social-media-december-2024-week1", split="train")



# Example: View the first few rows
#print(dataset)

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer # Corrected import
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # Corrected import
from sklearn.metrics import accuracy_score

import nltk
nltk.download('stopwords')

#printing the stopwords in english
print(stopwords.words('english'))


#Loading the data from a csv file

X_data = pd.read_csv("/content/x_en_dataset.csv",encoding = 'ISO-8859-1' )
#facebook_data
#Instagram_Data
#Checking number of rows and collumns
X_data.shape

#print first 5 rows
X_data.head()

#Counting the number of missing values in the dataset
X_data.isnull().sum()

#Checking the dsitribution of the target column
X_data['sentiment'].value_counts()



['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,count
sentiment,
0.14,71
0.17,65
0.07,65
0.18,61
0.04,60
...,...
0.89,2
-0.93,1
0.93,1


In [123]:


#Convert the main emotion Neutral to 1
X_data.replace({'main_emotion':{'neutral':0}},inplace=True)
#Convert the happy emotion to 1
X_data.replace({'main_emotion':['admiration','approval','caring','curiosity','desire','excitment','gratitude','joy','love','optimism','pride','realization','surprise']}, 1,inplace=True)
#Convert the negative emotion to -1
X_data.replace({'main_emotion':['anger','annoyance','confusion','disappointment','disapproval','disgust','embarrassment','excitement','fear','nervousness','remorse','sadness']},-1,inplace=True)


# Convert the column to string type to avoid TypeError
X_data['main_emotion'] = X_data['main_emotion'].astype(str)

/tmp/ipython-input-123-998666315.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_data.replace({'main_emotion':['anger','annoyance','confusion','disappointment','disapproval','disgust','embarrassment','excitement','fear','nervousness','remorse','sadness']},-1,inplace=True)


In [124]:
print(np.unique(X_data['main_emotion']))

['-1' '0' '1']


In [125]:
#Checking the dsitribution of the target column
X_data['main_emotion'].value_counts()

,count
main_emotion,
0,2848
1,1433
-1,719


In [126]:
port_stem = PorterStemmer()

def stemming(content):
        stemmed_content = re.sub('[^a-zA-Z]',' ', content)
        stemmed_content = stemmed_content.lower()
        stemmed_content = stemmed_content.split()
        stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
        stemmed_content = ' '.join(stemmed_content)
        return stemmed_content

X_data['stemmed_content']= X_data['original_text'].apply(stemming)

X_data.head()
print(X_data['stemmed_content'])
print(X_data['main_emotion'])

#seperating the data and label
X= X_data['stemmed_content'].values
Y= X_data['main_emotion'].values

print(X)
print(Y)

#splitting the dat into training and test

X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size =.4,stratify=Y,random_state=2)

print(X.shape,X_train.shape,X_test.shape)

print(X_train)
print(X_test)








0       make us realiz option see option deliv saturda...
1       grok ai repres scenario yeah miss tusk varahav...
2                                   pretti sure may undef
3       check r underr program languag ideal data anal...
4                     came car know go home smh piss cold
                              ...                        
4995                think enough cat health mainten parvo
4996                        sure mom ass get pretti messi
4997    joo bayden fodii wa ude njillu jaango altin an...
4998                                wing u dont mind give
4999    abbydphillip done play realli bad refere night...
Name: stemmed_content, Length: 5000, dtype: object
0        1
1        0
2        1
3        0
4        0
        ..
4995    -1
4996    -1
4997     0
4998     0
4999     0
Name: main_emotion, Length: 5000, dtype: object
['make us realiz option see option deliv saturday adapt dumb'
 'grok ai repres scenario yeah miss tusk varahavtar look badass say least'
 'pre

In [127]:
#COnverting the textual data to numerical data
vectorizer= TfidfVectorizer()

X_train= vectorizer.fit_transform(X_train)
X_test =vectorizer.transform(X_test)


print(X_train)
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 30627 stored elements and shape (3000, 8854)>
  Coords	Values
  (0, 4606)	0.15285461853201043
  (0, 2361)	0.32554115876204004
  (0, 7753)	0.31751749225139436
  (0, 1987)	0.20463836472055316
  (0, 273)	0.21334434452684503
  (0, 8046)	0.2547124904134316
  (0, 3618)	0.2547124904134316
  (0, 7965)	0.3107335788783804
  (0, 824)	0.23084570621784264
  (0, 6022)	0.2623854911164268
  (0, 3203)	0.17061157204896615
  (0, 6540)	0.32554115876204004
  (0, 4975)	0.30485708581874676
  (0, 7504)	0.32554115876204004
  (1, 6351)	0.40482448022171025
  (1, 798)	0.2640792235355425
  (1, 82)	0.24808287666351145
  (1, 8681)	0.3602061456062855
  (1, 4688)	0.22705744044882178
  (1, 464)	0.2903253978006086
  (1, 3247)	0.3602061456062855
  (1, 2666)	0.3602061456062855
  (1, 2514)	0.306709750845039
  (1, 4700)	0.29270121127893617
  (2, 5259)	0.5563761199559623
  :	:
  (2999, 639)	0.10321451083218995
  (2999, 7007)	0.11404894072116656
  (2999, 2180)	0.13

In [128]:
#Training the logistical Model


#logistic regression

model = LogisticRegression(max_iter=1000)
model.fit(X_train,Y_train)







LogisticRegression(max_iter=1000)

In [129]:
#Model Eval
#AccuracyScore

#Accuaracy score on training data
X_train_prediction= model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_prediction)




In [130]:
print("Accuracy Score on the training data : ",training_data_accuracy)

Accuracy Score on the training data :  0.8083333333333333


In [131]:
#Accuaracy score on test data
X_test_prediction= model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test,X_test_prediction)

print("Accuracy Score on the testing data : ",test_data_accuracy)


Accuracy Score on the testing data :  0.6605


In [132]:
import pickle

filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

#loading the saved model

loaded_model = pickle.load(open('/content/trained_model.sav','rb'))

X_new= X_test[200]
print(Y_test[200])


predicition = model.predict(X_new)
print (predicition)

if (predicition[-1]==-1):
        print('Negative Tweet')

else:
        print('Positive Tweet')

1
['1']
Positive Tweet


In [133]:
X_new= X_test[15]
print(Y_test[15])


predicition = model.predict(X_new)
print (predicition)

if (predicition[0]==-1):
        print('Negative Tweet')

else:
        print('Positive Tweet')

1
['0']
Positive Tweet


In [134]:
X_new= X_test[3]
print(Y_test[3])


predicition = model.predict(X_new)
print (predicition)

if (predicition[-1]==-1):
        print('Negative Tweet')

else:
        print('Positive Tweet')

0
['0']
Positive Tweet


In [135]:
X_new= X_test[17]
print(Y_test[17])


predicition = model.predict(X_new)
print (predicition)

if (predicition[-1]==-1):
        print('Negative Tweet')

else:
        print('Positive Tweet')

0
['0']
Positive Tweet
